In [93]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [94]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [95]:
data = pd.read_csv('Building_Energy_Benchmarking.csv')
data.head()

<ipython-input-95-68005734ec66>:1: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



,X,Y,PID,SSL,PMPROPERTYID,PROPERTYNAME,PMPARENTPROPERTYID,PARENTPROPERTYNAME,REPORTINGYEAR,REPORTSTATUS,ADDRESSOFRECORD,OWNEROFRECORD,WARD,REPORTEDADDRESS,CITY,STATE,POSTALCODE,YEARBUILT,PRIMARYPROPERTYTYPE_SELFSELECT,PRIMARYPROPERTYTYPE_EPACALC,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,ENERGYSTARSCORE,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERSCORE_MFPROPERTIES,WATERUSE_ALLWATERSOURCES_KGAL,METEREDAREAS_ENERGY,METEREDAREAS_WATER,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,GIS_LAST_MOD_DTTM,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER,UBID,LASTUPDATE,OBJECTID
0,-8.575303e+06,4.707303e+06,DC02220020,0222 0020,NaN,NaN,NaN,NaN,2021,Exempt from this year's disclosure,727 15TH ST NW,NATIONAL COMMUNITY REINVESTMENT COALITION,2.0,NaN,WASHINGTON,DC,20005,1985.0,NaN,NaN,46332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899610,-77.033254,218739.0,397115.369981,136862.520445,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/09/12 04:00:00+00,3541768
1,-8.575227e+06,4.707286e+06,DC02220027,0222 0027,NaN,NaN,NaN,NaN,2021,Exempt from this year's disclosure,1425 NEW YORK AVE NW,NY AVENUE TITLEHOLDER LLC,2.0,NaN,WASHINGTON,DC,20005,1991.0,NaN,NaN,158124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899493,-77.032576,300903.0,397174.190168,136849.566803,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/04/14 04:00:00+00,3541769
2,-8.575234e+06,4.707311e+06,DC02220027,0222 0027,NaN,NaN,NaN,NaN,2022,Exempt from this year's disclosure,1425 NEW YORK AVENUE NW,NY AVENUE TITLEHOLDER LLC,2.0,NaN,WASHINGTON,DC,20005,1991.0,NaN,NaN,158124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.899669,-77.032635,300903.0,397169.119971,136869.030539,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/04/14 04:00:00+00,3541770
3,-8.575281e+06,4.707161e+06,DC02230025,0223 0025,NaN,NaN,NaN,NaN,2012,No Report Received,1435 G ST NW,1440 NEW YORK AVENUE CORPORATION,2.0,NaN,WASHINGTON,DC,NaN,1934.0,NaN,NaN,228254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.898618,-77.033060,814459.0,397132.162500,136752.413602,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3541771
4,-8.575222e+06,4.707022e+06,DC02240824,0224 0824,NaN,NaN,NaN,NaN,2013,Exempt from this year's disclosure,1401 F ST NW,FSP HAMILTON SQUARE LLC,2.0,NaN,WASHINGTON,DC,20005,1999.0,NaN,NaN,269472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.897644,-77.032531,217997.0,397178.028700,136644.297802,2024/10/08 01:01:27+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024/06/06 04:00:00+00,3541772


In [96]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26577 entries, 0 to 26576
Data columns (total 73 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   X                               26577 non-null  float64
 1   Y                               26577 non-null  float64
 2   PID                             26577 non-null  object 
 3   SSL                             26577 non-null  object 
 4   PMPROPERTYID                    22224 non-null  float64
 5   PROPERTYNAME                    22142 non-null  object 
 6   PMPARENTPROPERTYID              937 non-null    object 
 7   PARENTPROPERTYNAME              937 non-null    object 
 8   REPORTINGYEAR                   26577 non-null  int64  
 9   REPORTSTATUS                    26577 non-null  object 
 10  ADDRESSOFRECORD                 26573 non-null  object 
 11  OWNEROFRECORD                   26571 non-null  object 
 12  WARD                            

In [97]:
data['PRIMARYPROPERTYTYPE_SELFSELECT'].unique()

array([nan, 'Office', 'Self-Storage Facility', 'Hotel',
       'Multifamily Housing', 'K-12 School', 'Residence Hall/Dormitory',
       'Medical Office', 'Mixed Use Property', 'Strip Mall',
       'Retail Store', 'Manufacturing/Industrial Plant',
       'Residential Care Facility', 'Other',
       'Hospital (General Medical & Surgical)',
       'Fitness Center/Health Club/Gym', 'Stadium (Open)',
       'Police Station', 'Other - Lodging/Residential',
       'Non-Refrigerated Warehouse', 'Other - Public Services',
       'Swimming Pool', 'Library', 'Pre-school/Daycare',
       'Repair Services (Vehicle, Shoe, Locksmith, etc)',
       'Social/Meeting Hall', 'Prison/Incarceration', 'Laboratory',
       'Fire Station', 'Adult Education', 'Food Sales',
       'Vehicle Repair Services', 'Single Family Home',
       'Bank/Financial Institution', 'Supermarket/Grocery Store',
       'College/University', 'Museum', 'Bank Branch', 'Financial Office',
       'Senior Care Community', 'Senior Living

In [98]:
university_property_types = ['College/University (Campus-Level)', 'College/University']
uni_data = data[data['PRIMARYPROPERTYTYPE_SELFSELECT'].isin(university_property_types)]

In [99]:
uni_data.shape

(506, 73)

In [100]:
uni_data.isna().sum()

,0
X,0
Y,0
PID,0
SSL,0
PMPROPERTYID,0
PROPERTYNAME,0
PMPARENTPROPERTYID,191
PARENTPROPERTYNAME,191
REPORTINGYEAR,0
REPORTSTATUS,0


In [101]:
uni_clean = uni_data.drop(columns=['PMPARENTPROPERTYID', 'PARENTPROPERTYNAME', 'CITY', 'STATE',
                                   'WATERSCORE_MFPROPERTIES', 'METEREDAREAS_WATER', 'UBID', 'ENERGYSTARSCORE'])
uni_clean.dropna(inplace=True)
# drop_na_columns = [col for col in uni_clean.columns if col != 'ENERGYSTARSCORE']
# uni_clean.dropna(subset=drop_na_columns, inplace=True)

In [102]:
uni_clean.head()

,X,Y,PID,SSL,PMPROPERTYID,PROPERTYNAME,REPORTINGYEAR,REPORTSTATUS,ADDRESSOFRECORD,OWNEROFRECORD,WARD,REPORTEDADDRESS,POSTALCODE,YEARBUILT,PRIMARYPROPERTYTYPE_SELFSELECT,PRIMARYPROPERTYTYPE_EPACALC,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERUSE_ALLWATERSOURCES_KGAL,METEREDAREAS_ENERGY,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,GIS_LAST_MOD_DTTM,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER,LASTUPDATE,OBJECTID
6489,-8.573755e+06,4.706375e+06,PM12413772,0491 3139,12413772.0,555 Penn,2021,In Compliance,555 PENNSYLVANIA AVE NW,JOHNS HOPKINS UNIVERSITY,2.0,"555 Pennsylvania Avenue, NW",20001,2007.0,College/University,Other - Technology/Science,389719,567029.0,49.2,50.4,121.6,122.8,2314.5,4.1,3479.1,Whole Property,52665.700000,0.0,38.893125,-77.019350,297641.0,398321.340120,136142.277553,2024/10/08 01:01:27+00,0.0,0.0,0.0,0.0,6636294.7,1153591.9,1139527.4,737032.0,239922.2,55325.5,10511.2,5620.1,6782.9,15285.0,85281.5,806749.1,1010937.0,2380263.7,2272256.6,2551768.7,2309788.2,2005702.5,1745792.3,1630943.0,1528916.0,1483575.6,1462149.9,1480087.8,1791793.2,2024/09/26 04:00:00+00,3575441
6491,-8.573755e+06,4.706375e+06,PM12413772,0491 3139,12413772.0,555 Penn,2023,Incomplete Report,555 PENNSYLVANIA AVE NW,THE JOHNS HOPKINS UNIVERSITY,2.0,"555 Pennsylvania Avenue, NW",20001,2007.0,College/University,Mixed Use Property,389719,604762.5,95.7,96.7,255.1,256.2,4924.9,8.1,2640.6,Whole Property,44464.800000,0.0,38.893125,-77.019350,297641.0,398321.340120,136142.277553,2024/10/08 01:01:27+00,0.0,0.0,0.0,0.0,15657705.5,1362353.9,1218638.8,1092374.6,319677.8,32908.3,8215.1,4863.0,6813.6,18404.8,27549.4,145089.9,209592.6,4913063.1,3688806.0,3500066.2,3080491.1,2854650.7,3219963.8,4556421.9,5294404.7,5093514.0,5518081.3,5972447.2,5732181.3,2024/09/26 04:00:00+00,3575443
6604,-8.572559e+06,4.707432e+06,PM1309871,0676 0114,1309871.0,801 North Capitol Street,2017,In Compliance,801 NORTH CAPITOL ST NE,ASB 801 NORTH CAPITOL LLC,6.0,"801 North Capitol Street, NE",20002,1966.0,College/University,College/University,98980,120921.0,53.8,54.4,161.0,161.6,695.2,5.7,1600.6,Whole Building,4519.882241,0.0,38.900510,-77.008608,79625.0,399253.312265,136961.994547,2024/10/08 01:01:27+00,0.0,0.0,0.0,0.0,1773186.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024/06/19 04:00:00+00,3575556
6605,-8.572559e+06,4.707431e+06,PM1309871,0676 0114,1309871.0,801 North Capitol Street,2018,In Compliance,801 NORTH CAPITOL ST NE,ASB 801 NORTH CAPITOL LLC,6.0,"801 North Capitol Street, NE",20002,1966.0,College/University,College/University,98980,120921.0,10.9,70.0,17.1,183.2,86.8,0.7,1495.0,Whole Property,9247.307959,0.0,38.900503,-77.008606,79625.0,399253.510118,136961.167046,2024/10/08 01:01:27+00,0.0,0.0,0.0,0.0,2221630.3,467779.9,312391.2,84946.8,548416.3,201955.8,1327.2,26.5,0.0,447.0,7218.6,247899.9,181956.1,511956.3,464394.1,465090.9,441370.0,449066.8,445012.9,474248.9,1761592.2,1245231.6,402871.4,6914.7,376362.2,2024/06/19 04:00:00+00,3575557
6606,-8.572559e+06,4.707431e+06,PM1309871,0676 0114,1309871.0,801 North Capitol Street,2019,In Compliance,801 NORT

In [103]:
uni_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 368 entries, 6489 to 26221
Data columns (total 65 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   X                               368 non-null    float64
 1   Y                               368 non-null    float64
 2   PID                             368 non-null    object 
 3   SSL                             368 non-null    object 
 4   PMPROPERTYID                    368 non-null    float64
 5   PROPERTYNAME                    368 non-null    object 
 6   REPORTINGYEAR                   368 non-null    int64  
 7   REPORTSTATUS                    368 non-null    object 
 8   ADDRESSOFRECORD                 368 non-null    object 
 9   OWNEROFRECORD                   368 non-null    object 
 10  WARD                            368 non-null    float64
 11  REPORTEDADDRESS                 368 non-null    object 
 12  POSTALCODE                      368 

In [104]:
uni_clean.describe()

,X,Y,PMPROPERTYID,REPORTINGYEAR,WARD,YEARBUILT,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERUSE_ALLWATERSOURCES_KGAL,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER,OBJECTID
count,3.680000e+02,3.680000e+02,3.680000e+02,368.000000,368.000000,368.000000,3.680000e+02,3.680000e+02,368.000000,368.000000,368.000000,368.000000,368.000000,368.00000,3.680000e+02,3.680000e+02,3.680000e+02,368.000000,368.000000,368.000000,368.000000,368.000000,3.680000e+02,3.680000e+02,3.680000e+02,368.000000,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02,3.680000e+02
mean,-8.575216e+06,4.710208e+06,4.250716e+06,2016.766304,2.989130,1966.154891,3.606711e+05,4.987533e+05,76.823641,77.913859,155.288315,156.011685,4446.816766,6.35788,4.193371e+04,3.215631e+05,2.717689e+05,38.919918,-77.032478,308971.239130,397183.495728,139117.319566,-1.366833e+06,1.771722e+05,-9.929268e+05,12906.658424,7.907764e+06,1.878940e+06,1.588969e+06,1.392229e+06,9.010797e+05,5.899469e+05,5.068861e+05,4.739080e+05,4.837950e+05,5.457165e+05,7.691196e+05,1.251682e+06,1.465494e+06,9.671285e+05,9.258188e+05,9.643817e+05,9.605336e+05,1.080892e+06,1.199656e+06,1.363209e+06,1.407089e+06,1.303620e+06,1.132520e+06,9.928597e+05,9.498186e+05,3.586394e+06
std,3.428750e+03,2.391641e+03,2.734404e+06,2.961102,1.630731,37.063984,5.644931e+05,9.490726e+05,107.390965,107.774583,171.819531,172.064154,10102.973312,7.76471,2.171509e+05,9.089774e+05,1.716503e+06,0.016715,0.030801,162742.541599,2670.774870,1855.624536,1.701440e+07,1.611724e+06,1.565415e+07,72653.273170,1.807768e+07,7.076439e+06,5.982621e+06,5.030614e+06,3.439285e+06,2.429983e+06,2.216172e+06,2.109892e+06,2.122371e+06,2.401427e+06,2.977101e+06,4.603528e+06,5.144722e+06,3.354602e+06,3.237914e+06,3.362148e+06,3.381676e+06,3.846553e+06,4.276352e+06,4.859088e+06,5.087463e+06,4.704311e+06,4.068128e+06,3.486815e+06,3.282999e+06,5.476168e+03
min,-8.582171e+06,4.706375e+06,1.309871e+06,2012.000000,1.000000,1867.000000,0.000000e+00,6.700000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000e+00,0.000000e+00,0.000000e+00,38.893125,-77.094952,76086.000000,391768.729640,136142.277553,-1.959899e+08,0.000000e+00,-1.742818e+08,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.575441e+06
25%,-8.577224e+06,4.708106e+06,3.483049e+06,2014.000000,2.000000,1946.000000,9.898000e+04,7.260000e+04,10.975000,11.450000,13.025000,12.050000,100.450000,0.55000,1.272450e+03,4.236350e+01,0.000000e+00,38.905226,-77.050514,242386.000000,395618.240530,137487.797372,0.000000e+00,0.000000e+00,0.00000

In [105]:
def clean_address(address):
  # To clean the address column
  address = address.upper()
  address = re.sub(r'\bST\b', 'STREET', address)
  address = re.sub(r'\bAVE\b', 'AVENUE', address)
  address = re.sub(r'\bAV\b', 'AVENUE', address)
  address = re.sub(r'\bRD\b', 'ROAD', address)
  address = re.sub(r'\bBLVD\b', 'BOULEVARD', address)
  address = address.lstrip('0')
  # Adding space between street name and directional indicators if missing
  address = re.sub(r'(\w)(NW|NE|SW|SE)', r'\1 \2', address)

  address = address.replace("MASSACHU SETTS", "MASSACHUSETTS")
  address = address.replace("CON NECTICUT", "CONNECTICUT")
  address = address.replace("RE SERVOIR", "RESERVOIR")
  return address

def add_missing_quadrants(df, column):
    # Compile regex for quadrant detection
    quadrant_pattern = re.compile(r'\b(NE|NW|SE|SW)\b$')

    # Create a dictionary to map base addresses to their quadrants
    base_address_map = {}
    for address in df[column].unique():
        base_address = re.sub(r'\b(NE|NW|SE|SW)\b$', '', address).strip()
        quadrant = quadrant_pattern.search(address)
        if quadrant:
            base_address_map[base_address] = quadrant.group()

    # Function to append missing quadrants
    def append_quadrant(address):
        base_address = re.sub(r'\b(NE|NW|SE|SW)\b$', '', address).strip()
        if not quadrant_pattern.search(address) and base_address in base_address_map:
            return f"{address} {base_address_map[base_address]}"
        return address

    # Apply function to DataFrame column
    df[column] = df[column].apply(append_quadrant)
    return df

In [106]:
# Cleaning the address column
uni_clean['ADDRESSOFRECORD'] = uni_clean['ADDRESSOFRECORD'].apply(clean_address)
uni_clean = add_missing_quadrants(uni_clean, 'ADDRESSOFRECORD')
#uni_clean = add_missing_directional(uni_clean, 'ADDRESSOFRECORD')

In [107]:
print(uni_clean['ADDRESSOFRECORD'].nunique())
uni_clean['ADDRESSOFRECORD'].unique()

65


array(['555 PENNSYLVANIA AVENUE NW', '801 NORTH CAPITOL STREET NE',
       '4TH STREET NE', '125 MICHIGAN AVENUE NE', '2100 FOXHALL ROAD NW',
       '1717 MASSACHUSETTS AVENUE NW', '101 F STREET NW',
       '111 G STREET NW', '550 1ST STREET NW', '2001 WISCONSIN AVENUE NW',
       '2013 H STREET NW', '800 21ST STREET NW', '805 21ST STREET NW',
       '600 22ND STREET NW', '1922 F STREET NW', '2033 G STREET NW',
       '714 21ST STREET NW', '2201 G STREET NW', '801 22ND STREET NW',
       '2300 I STREET NW', '1740 MASSACHUSETTS AVENUE NW',
       '1619 MASSACHUSETTS AVENUE NW', '4801 MASSACHUSETTS AVENUE NW',
       '950 NEW HAMPSHIRE AVENUE', '4400 MASSACHUSETTS AVENUE NW',
       '3515 PROSPECT STREET NW', '3800 RESERVOIR ROAD',
       '3600 M STREET NW', '2125 G STREET NW', '1251 6TH STREET NE',
       '800 FLORIDA AVENUE NE', '2900 VAN NESS STREET NW',
       '2130 H STREET', '3700 O STREET NW', '1608 RHODE ISLAND AVENUE NW',
       '2004 G STREET NW', '2000 G STREET NW', '401 MICHI

In [108]:
owner_replacements = {
    'THE GEORGE WASHINGTON UNIVERSITY': 'GEORGE WASHINGTON UNIVERSITY',
    'PRESIDENT & DIRECTORS OF GEORGETOWN UNIVERSITY': 'GEORGETOWN UNIVERSITY',
    'THE PRESIDENT AND DIRECTORS OF GEORGETOWN COLLEGE': 'GEORGETOWN UNIVERSITY',
    'GEORGETOWN COLLEGE LAW CENTER': 'GEORGETOWN UNIVERSITY',
    'JOHNS HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    'THE JOHN HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    'THE JOHNS HOPKINS UNIVERSITY': 'JOHN HOPKINS UNIVERSITY',
    'THE HOWARD UNIVERSITY INC': 'HOWARD UNIVERSITY',
    'HOWARD UNIVERSITY (LAND ONLY)': 'HOWARD UNIVERSITY',
    'THE HOWARD UNIVERSITY': 'HOWARD UNIVERSITY',
    'HOWARD DORMITORY HOLDINGS 1 LLC' : 'HOWARD UNIVERSITY',
    'ASB 801 NORTH CAPITOL LLC' : 'HOWARD UNIVERSITY',
    'ASB 601 NORTH CAPITOL LLC' : 'HOWARD UNIVERSITY',
    'GREEN HARRIS LLC' : 'GEORGETOWN UNIVERSITY',
    "JEMAL'S CAYRE CARBARN LLC" : 'GEORGETOWN UNIVERSITY',
    'JESUIT COMMUNITY AT GEORGETOWN UNIV INC' : 'GEORGETOWN UNIVERSITY',
    'THE REGENTS OF THE UNIVERSITY OF CALIFORNIA' : 'UNIVERSITY OF CALIFORNIA WASHINGTON CENTER',
    'THEOLOGICAL COLLEGE INC' : 'THEOLOGICAL COLLEGE',
    'CATHOLIC UNIVERSITY OF AMERICA' : 'THE CATHOLIC UNIVERSITY OF AMERICA',
    'TRUSTEES FOR HARVARD UNIVERSITY' : 'HARDWARD UNIVERSITY',
    'DISTRICT OF COLUMBIA' : 'UNIVERSITY OF THE DISTRICT OF COLUMBIA'
}

def clean_owner(owner):
    if owner in owner_replacements:
        owner = owner_replacements[owner]
    return owner

In [109]:
uni_clean['OWNEROFRECORD'] = uni_clean['OWNEROFRECORD'].apply(clean_owner)

In [111]:
# Dropping some entries that are not College/University
uni_clean = uni_clean[~uni_clean['OWNEROFRECORD'].isin(['WESLEY THEOLOGICAL SEMINARY OF METHODIST CHURCH',
                                                        'MISSIONARY SOCIETY OF ST PAUL TH EAPOSTLE IN THE STATE OF NY',
                                                        "JEMAL'S WTU LLC"
                                                        ])]

In [112]:
print(uni_clean['OWNEROFRECORD'].nunique())
uni_clean['OWNEROFRECORD'].unique()

14


array(['JOHN HOPKINS UNIVERSITY', 'HOWARD UNIVERSITY', 'TRINITY COLLEGE',
       'GEORGE WASHINGTON UNIVERSITY', 'GEORGETOWN UNIVERSITY',
       'AMERICAN UNIVERSITY', 'GALLAUDET UNIVERSITY',
       'UNIVERSITY OF CALIFORNIA WASHINGTON CENTER',
       'THEOLOGICAL COLLEGE', 'THE CATHOLIC UNIVERSITY OF AMERICA',
       'NEW YORK UNIVERSITY', 'HARDWARD UNIVERSITY', 'JEMALS WTU LLC',
       'UNIVERSITY OF THE DISTRICT OF COLUMBIA'], dtype=object)

In [ ]:
uni_clean.head()

In [114]:
fig = px.scatter_mapbox(uni_clean, lat="LATITUDE", lon="LONGITUDE", color="OWNEROFRECORD",
                        hover_name="PROPERTYNAME", hover_data=["ADDRESSOFRECORD", "REPORTINGYEAR"],
                        title="Map of University Buildings, Washington, DC")
fig.update_layout(mapbox_style="carto-positron")
fig.show()

In [115]:
def convert_to_mmbtu(df, features):
    # Converts selected features in the dataframe to MMBTU using appropriate conversion factors.
    conversion_factors = {
        'SOURCEEUI_KBTU_FT': 0.001,  # KBTU to MMBTU
        'TOTGHGEMISSIONS_METRICTONSCO2E': 0.003412,  # GHG emissions to MMBTU
        'WATERUSE_ALLWATERSOURCES_KGAL': 0.000003412,  # Gallons of water to MMBTU
        'ELECTRICITYUSE_GRID_KWH': 0.003412,  # kWh to MMBTU
        'NATURALGASUSE_THERMS': 0.1,  # Therms to MMBTU
        'FUELOILANDDIESELFUELUSEKBTU': 0.001  # KBTU to MMBTU
    }

    # Convert each feature
    for feature in features:
        if feature in conversion_factors:
            df[feature] = df[feature] * conversion_factors[feature]

    return df

def plot_university_efficiency(df, features, year=None):
    # Plots a stacked bar chart comparing energy usage across universities for selected features in a given year.
    # Set the year to the most recent if not specified
    if year is None:
        year = df['REPORTINGYEAR'].max()

    # Filter the data for the specified year
    df_year = df[df['REPORTINGYEAR'] == year]

    # Convert features to MMBTU
    df_year = convert_to_mmbtu(df_year, features)
    df_year[features] = df_year[features].apply(pd.to_numeric, errors='coerce')  # Ensure columns are numeric
    df_year = df_year.dropna(subset=features)

    # Stacked bar chart: Energy usage for each university across selected features
    stacked_bar = px.bar(
        df_year,
        x='OWNEROFRECORD',  # Replace 'PROPERTYNAME' with the actual column name for university names
        y=features,
        title=f"Energy Usage for Each University in {year} (MMBTU)",
        labels={'value': 'Energy Usage (MMBTU)', 'PROPERTYNAME': 'University'},
        text_auto=True
    )

    # Update layout for better readability
    stacked_bar.update_layout(
        barmode='stack',
        xaxis_title="University",
        yaxis_title="Energy Usage (MMBTU)",
        plot_bgcolor='white',  # Clean white background for a professional look
        paper_bgcolor='white',  # White background for the entire plot
        font=dict(family="Arial", size=12, color="black")
    )

    return stacked_bar

In [116]:
features = ['SOURCEEUI_KBTU_FT', 'TOTGHGEMISSIONS_METRICTONSCO2E', 'WATERUSE_ALLWATERSOURCES_KGAL', 'ELECTRICITYUSE_GRID_KWH', 'NATURALGASUSE_THERMS', 'FUELOILANDDIESELFUELUSEKBTU']
plot_university_efficiency(uni_clean, features)

<ipython-input-115-662987dc8937>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-115-662987dc8937>:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
# gw_data = uni_clean[uni_clean['OWNEROFRECORD'].str.contains("GEORGE WASHINGTON UNIVERSITY")]

In [63]:
# print(gw_data.info())
# gw_data.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, 7209 to 22899
Data columns (total 66 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   X                               72 non-null     float64
 1   Y                               72 non-null     float64
 2   PID                             72 non-null     object 
 3   SSL                             72 non-null     object 
 4   PMPROPERTYID                    72 non-null     float64
 5   PROPERTYNAME                    72 non-null     object 
 6   REPORTINGYEAR                   72 non-null     int64  
 7   REPORTSTATUS                    72 non-null     object 
 8   ADDRESSOFRECORD                 72 non-null     object 
 9   OWNEROFRECORD                   72 non-null     object 
 10  WARD                            72 non-null     float64
 11  REPORTEDADDRESS                 72 non-null     object 
 12  POSTALCODE                      72 no

,X,Y,PMPROPERTYID,REPORTINGYEAR,WARD,YEARBUILT,TAXRECORDFLOORAREA,REPORTEDBUILDINGGROSSFLOORAREA,ENERGYSTARSCORE,SITEEUI_KBTU_FT,WEATHERNORMALZEDSITEEUI_KBTUFT,SOURCEEUI_KBTU_FT,WEATHERNORMALZEDSOUREUI_KBTUFT,TOTGHGEMISSIONS_METRICTONSCO2E,TOTGHGEMISSINTENSITY_KGCO2EFT,WATERUSE_ALLWATERSOURCES_KGAL,NATURALGASUSE_THERMS,FUELOILANDDIESELFUELUSEKBTU,LATITUDE,LONGITUDE,ADDRESSID,XCOORD,YCOORD,DISTRCHILLEDWATER_KBTU,DISTRHOTWATER_KBTU,DISTRSTEAM_KBTU,ELECTRICITYUSE_RENEWABLE_KWH,ELECTRICITYUSE_GRID_KWH,NATURALGAS_KBTU_JANUARY,NATURALGAS_KBTU_FEBRUARY,NATURALGAS_KBTU_MARCH,NATURALGAS_KBTU_APRIL,NATURALGAS_KBTU_MAY,NATURALGAS_KBTU_JUNE,NATURALGAS_KBTU_JULY,NATURALGAS_KBTU_AUGUST,NATURALGAS_KBTU_SEPTEMBER,NATURALGAS_KBTU_OCTOBER,NATURALGAS_KBTU_NOVEMBER,NATURALGAS_KBTU_DECEMBER,ELECTRICITYUSE_KBTU_JANUARY,ELECTRICITYUSE_KBTU_FEBRUARY,ELECTRICITYUSE_KBTU_MARCH,ELECTRICITYUSE_KBTU_APRIL,ELECTRICITYUSE_KBTU_MAY,ELECTRICITYUSE_KBTU_JUNE,ELECTRICITYUSE_KBTU_JULY,ELECTRICITYUSE_KBTU_AUGUST,ELECTRICITYUSE_KBTU_SEPTEMBER,ELECTRICITYUSE_KBTU_OCTOBER,ELECTRICITYUSE_KBTU_NOVEMBER,ELECTRICITYUSE_KBTU_DECEMBER,OBJECTID
count,7.200000e+01,7.200000e+01,7.200000e+01,72.000000,72.000000,72.000000,72.000000,7.200000e+01,1.0,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,7.200000e+01,7.200000e+01,72.000000,72.000000,72.000000,72.000000,72.000000,72.0,72.0,7.200000e+01,72.0,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,72.000000,72.000000,72.000000,7.200000e+01,72.000000,7.200000e+01,7.200000e+01,7.200000e+01,72.000000,72.000000,72.000000,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,7.200000e+01,72.000000,7.200000e+01
mean,-8.577843e+06,4.707843e+06,4.684381e+06,2016.430556,2.194444,1974.291667,210323.597222,2.604046e+05,55.0,150.618056,151.319444,290.875000,290.473611,4629.966250,12.137500,11311.259722,4.007065e+05,6.638341e+05,38.903386,-77.056071,408941.138889,395136.541321,137282.833943,0.0,0.0,1.803812e+05,0.0,5.855358e+06,1.878001e+05,1.558208e+05,1.552281e+05,2.092525e+05,49897.876389,30307.718056,30051.162500,4.963514e+04,41503.286111,7.934544e+04,1.235718e+05,1.540543e+05,97694.191667,91735.645833,99784.259722,1.018649e+05,1.163276e+05,1.288356e+05,1.384219e+05,1.488612e+05,1.443424e+05,1.204598e+05,9.779614e+04,95277.990278,3.581511e+06
std,1.909999e+03,1.121352e+03,4.629457e+06,2.842467,0.398550,32.180143,151262.708022,2.318951e+05,NaN,200.429936,200.609892,282.173919,282.618755,7709.073423,13.223618,16432.436099,9.335536e+05,1.821690e+06,0.007839,0.017158,277095.931339,1487.535910,871.195107,0.0,0.0,9.260512e+05,0.0,7.377621e+06,5.074343e+05,4.203255e+05,4.542431e+05,1.124340e+06,145611.669736,90800.277741,92396.354203,2.372791e+05,120366.348966,2.347988e+05,3.454393e+05,4.143215e+05,238411.458483,227205.660824,247891.313444,2.621649e+05,3.112286e+05,3.403052e+05,3.624089e+05,3.907113e+05,3.749864e+05,3.142862e+05,2.476722e+05,232758.427172,3.761227e+03
min,-8.581848e+06,4.706766e+06,2.265015e+06,2012.000000,2.000000,1893.000000,0.000000,2.761100e+04,55.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,38.895854,-77.092046,224574.000000,392017.479829,136445.864381,0.0,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,3.576161e+06
25%,-8.577255e+06,4.707196e+06,2.266110e+06,2014.000000,2.000000,1964.500000,102323.250000,1.194700e+05,55.0,73.000000,73.875000,149.550000,151.450000,956.650000,5.500000,1539.025000,9.472437e+03,0.000000e+00,38.898860,-77.050792,240947.000000,395594.118300,136779.772277,0.0,0.0,0.000000e+00,0.0,1.962027e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+0

In [117]:
def plot_building_efficiency(df, features, university_name, year=None):
    # Plots a stacked bar chart comparing energy usage for buildings within a specific university
    # for selected features in a given year.
    # Filter data for the specified university
    df_uni = df[df['OWNEROFRECORD'] == university_name]

    # Set the year to the most recent if not specified
    if year is None:
        year = df_uni['REPORTINGYEAR'].max()

    # Filter the data for the specified year
    df_year = df_uni[df_uni['REPORTINGYEAR'] == year]

    # Convert features to MMBTU
    df_year = convert_to_mmbtu(df_year, features)
    df_year[features] = df_year[features].apply(pd.to_numeric, errors='coerce')  # Ensure columns are numeric
    df_year = df_year.dropna(subset=features)

    # Stacked bar chart: Energy usage for each building within the specified university across selected features
    stacked_bar = px.bar(
        df_year,
        x='PROPERTYNAME',  # Replace 'PROPERTYNAME' with the actual column name for building names
        y=features,
        title=f"Energy Usage for Each Building in {year} (MMBTU) - {university_name}",
        labels={'value': 'Energy Usage (MMBTU)', 'PROPERTYNAME': 'Building'},
        text_auto=True
    )

    # Update layout for better readability
    stacked_bar.update_layout(
        barmode='stack',
        xaxis_title="Building",
        yaxis_title="Energy Usage (MMBTU)",
        plot_bgcolor='white',  # Clean white background for a professional look
        paper_bgcolor='white',  # White background for the entire plot
        font=dict(family="Arial", size=12, color="black")
    )

    return stacked_bar

In [118]:
plot_building_efficiency(uni_clean, features, "GEORGE WASHINGTON UNIVERSITY", 2017)

<ipython-input-115-662987dc8937>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-117-c42299ea5546>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [119]:
def plot_building_time_series(df, feature, university_name):
    # Plots a time series chart comparing energy usage for a specific feature across university buildings over the years
    # for a specific university.
    # Filter data for the specified university
    df_uni = df[df['OWNEROFRECORD'] == university_name]

    # Ensure the feature is numeric
    df_uni[feature] = pd.to_numeric(df_uni[feature], errors='coerce')

    # Drop rows where the feature is NaN
    df_uni = df_uni.dropna(subset=[feature])

    # Time series plot: Energy usage over years for each university building
    fig = px.line(
        df_uni,
        x='REPORTINGYEAR',  # Time axis (Year)
        y=feature,  # Feature to plot
        color='PROPERTYNAME',  # Different lines for each building
        title=f"Energy Usage by Year for Each Building (Feature: {feature}) - {university_name}",
        labels={'REPORTINGYEAR': 'Year', feature: f'Energy Usage ({feature})', 'PROPERTYNAME': 'University Building'},
        markers=True  # Adds markers at each point on the line for better visibility
    )

    # Update layout for better readability
    fig.update_layout(
        xaxis_title="Year",
        yaxis_title=f"Energy Usage ({feature})",
        plot_bgcolor='white',  # Clean white background for a professional look
        paper_bgcolor='white',  # White background for the entire plot
        font=dict(family="Arial", size=12, color="black"),
        showlegend=True  # Show legend for interactivity
    )

    return fig

In [120]:
features = ['SOURCEEUI_KBTU_FT', 'TOTGHGEMISSIONS_METRICTONSCO2E', 'WATERUSE_ALLWATERSOURCES_KGAL', 'ELECTRICITYUSE_GRID_KWH', 'NATURALGASUSE_THERMS', 'FUELOILANDDIESELFUELUSEKBTU']
for feature in features:
  fig = plot_building_time_series(uni_clean, feature, "GEORGE WASHINGTON UNIVERSITY")
  fig.show()

<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-119-271ef4929e1f>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
def plot_eui_buildings_on_map(df, year, university_name, threshold=180.6):
    # Plots university buildings on a Carto Positron map, color-coding them based on SOURCEEUI_KBTU_FT.
    # Filter data for the specified university and year
    df_filtered = df[(df['OWNEROFRECORD'] == university_name) & (df['REPORTINGYEAR'] == year)].copy()

    # Check if necessary columns are present
    if 'LONGITUDE' not in df.columns or 'LATITUDE' not in df.columns or 'SOURCEEUI_KBTU_FT' not in df.columns:
        raise ValueError("The dataset must contain 'Longitude', 'Latitude', and 'SOURCEEUI_KBTU_FT' columns.")

    # Color-code buildings based on the threshold value of SOURCEEUI_KBTU_FT
    df_filtered['Color'] = df_filtered['SOURCEEUI_KBTU_FT'].apply(lambda x: "Source EUI above 180.6" if x > threshold else "Source EUI below or equal to 180.6")

    # Create the map plot
    fig = px.scatter_mapbox(
        df_filtered,
        lat="LATITUDE",
        lon="LONGITUDE",
        color="Color",
        hover_name="PROPERTYNAME",
        hover_data={"SOURCEEUI_KBTU_FT": True, "Color": False},
        title=f"{university_name} Buildings in {year} (Red: > {threshold} SOURCEEUI_KBTU_FT, Green: ≤ {threshold})",
        zoom=13,
        height=600,
        color_discrete_map={"Source EUI above 180.6": "red", "Source EUI below or equal to 180.6": "green"}  # Define the colors for the markers
    )

    # Update map layout for Carto Positron style
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_center={"lat": df_filtered["LATITUDE"].mean(), "lon": df_filtered["LONGITUDE"].mean()},
        margin={"r": 0, "t": 50, "l": 0, "b": 0}
    )

    return fig


In [123]:
fig = plot_eui_buildings_on_map(gw_data, 2017, "GEORGE WASHINGTON UNIVERSITY")
fig.show()